# Song Recommender - Billboard Top 100

In [1]:
import pandas as pd

## Web Scraping 

First step in the song recommender project was scraping data from the Billboard Hot 100 List.
<br> 
We wrote a function that can be reused and will store the up to date data in a .csv file. 

In [2]:
def scrape_billboard ():
    
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    
    url = "https://www.billboard.com/charts/hot-100/"
    
    response = requests.get(url)
    
    response.content
    
    status = response.status_code
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    artists = []
    songs = []

    for number, item in enumerate(soup.select(".a-no-trucate"), start=0):
        if number % 2 == 0:
            songs.append(item.get_text())
        else:
            artists.append(item.get_text())
            
    artists_names = []
    song_names = []
    
    for i in artists: 
        artists_names.append(i.strip())
    
    for i in songs: 
        song_names.append(i.strip())
    
    billboard_100 = pd.DataFrame({'song':song_names,'artist':artists_names})
    
    billboard_100.song = billboard_100.song.str.lower()
    billboard_100.artist = billboard_100.artist.str.lower()
    
    billboard_100.to_csv('Data/Scraped_Billboard_Top_100.csv', index=False)
    
    return billboard_100

In [3]:
scrape_billboard()

,song,artist
0,heat waves,glass animals
1,stay,the kid laroi & justin bieber
2,super gremlin,kodak black
3,abcdefu,gayle
4,we don't talk about bruno,"carolina gaitan, mauro castillo, adassa, rhenz..."
...,...,...
95,by your side,rod wave
96,waiting on a miracle,stephanie beatriz
97,i hate youngboy,youngboy never broke again
98,city of gods,"fivio foreign, kanye west & alicia keys"


## Recommender - Top 100

This recommender uses the Top 100 Charts scraped from Billboard. If user input is a song that is currently on the Billboard Top Chart List, the recommender will give another song from the list, otherwise there will be no recommendation given. <br> Later this recommender will be a part of the Spotify song recommender.

In [4]:
billboard_100 = pd.read_csv("Data/Scraped_Billboard_Top_100.csv")

In [5]:
def is_song_in_top_100(df):
    
    from pyjarowinkler import distance
    
    user_input = input("Please enter a song name here: ")
    user_input = user_input.lower()
    

    for i in list(df.song):
        
        dist = distance.get_jaro_distance(i, user_input)
        
        if dist == 1:
            recommendation = df[df.song != user_input].sample(n=1)
            title = recommendation.iloc[0,0]
            artist = recommendation.iloc[0,1]
            return print('Your song is in the Billboard Top 100 Songs!','\n','Here is your recommendation:',title, 'by', artist)
        
        
        elif dist >= 0.90 and dist < 1:
            answer = input('Are you sure your input was correct? You entered '+ user_input + ', did you mean '+ i + ' ? (yes/no) ').lower()
            
            if answer == 'yes':
                recommendation = df[df.song != user_input].sample(n=1)
                title = recommendation.iloc[0,0]
                artist = recommendation.iloc[0,1]
                return print('\n','Your song is in the Billboard Top 100 Songs!','\n','Here is your recommendation:',title, 'by', artist)
            
            elif answer == 'no':
                answer2 = input('Do you want to try again? (yes/no)').lower() 
                if answer2 == 'yes':
                    return is_song_in_top_100(df)
                elif answer2 == 'no':
                    return print('\n','Your song is not in the Billboard Top 100 Songs, sorry!')
                else:
                    return print('Your response was neither yes nor no, please start again.') + is_song_in_top_100(df)
            else:
                    return print('Your response was neither yes nor no, please start again.') + is_song_in_top_100(df)
                
    return print('Your song is not in the Billboard Top 100 Songs, sorry!')

In [9]:
is_song_in_top_100(billboard_100)

Please enter a song name here: tay
Are you sure your input was correct? You entered tay, did you mean stay ? (yes/no) no
Do you want to try again? (yes/no)yes
Please enter a song name here: stay
Your song is in the Billboard Top 100 Songs! 
 Here is your recommendation: knife talk by drake featuring 21 savage & project pat
